# 10 Pinecone을 활용한 RAG 파이프라인 구축

In [9]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
   chunk_size=1500,
   chunk_overlap=200,
   separators=[
       "\n\n",    # 빈 줄
       "\n",      # 줄바꿈
       " ",       # 공백
       ".",       # 마침표
       ",",       # 쉼표
       "\u200b",  # 폭 없는 공백
       "\uff0c",  # 전각 쉼표
       "\u3001",  # 한중일 쉼표
       "\uff0e",  # 전각 마침표
       "\u3002",  # 한중일 마침표
       "",
   ],
)

# 워드 문서를 로드하고 설정한 text_splitter로 분할한다.
loader = Docx2txtLoader('../documents/law_markdown.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [10]:
# 필요한 라이브러리들을 가져온다
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

# 환경변수 파일(.env)을 불러온다
load_dotenv()

# Upstage의 SOLAR 임베딩 모델을 초기화한다
embedding = UpstageEmbeddings(
    model='solar-embedding-1-large'  
)


In [ ]:
from langchain_chroma import Chroma

# Chroma를 활용한 기존 코드 
vector_store = Chroma.from_documents(
            documents=document_list, 
            embedding=embedding, 
            collection_name='tax-markdown', 
            persist_directory="./tax-markdown")

In [ ]:
import os
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore.from_documents(
                document_list, 
                embedding, 
                index_name="house-tax-index")

# retriever를 설정하고 상위 3개 결과를 반환하도록 한다
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

In [17]:
from langchain import hub

rag_prompt = hub.pull("rlm/rag-prompt")

# 여러 문서들을 하나의 문자열로 결합하는 헬퍼 함수다.
# 각 문서는 두 줄의 개행으로 구분되어 LLM이 문맥을 더 잘 파악할 수 있다.
def format_docs(docs):
   return "\n\n".join(doc.page_content for doc in docs)


In [ ]:
#from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

# llm = ChatOpenAI(model="gpt-4o", temperature=0) # 기존 gpt-4o는 주석 처리한다.
llm = ChatAnthropic(model="claude-opus-4-20250514", temperature=0)


In [34]:
# LCEL의 기본 컴포넌트들을 가져온다
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# RAG 체인을 구성한다
# 1. retriever로 문서를 가져와서 포맷팅한다
# 2. 프롬프트에 문서와 질문을 전달한다
# 3. LLM으로 답변을 생성한다
# 4. 문자열로 파싱한다
tax_base_chain = (
   {"context": retriever | format_docs, "question": RunnablePassthrough()}
   | rag_prompt
   | llm
   | StrOutputParser()
)

tax_base_question = "주택에 대한 종합부동산세 과세표준을 계산하는 방법을 수식으로 표현해서 수식만 반환해주세요. 부연설명을 하지 말아주세요"
tax_base_response = tax_base_chain.invoke(tax_base_question)

In [35]:
# 필요한 라이브러리들을 가져온다
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# LangChain Hub에서 RAG 프롬프트 템플릿을 가져온다
rag_prompt = hub.pull("rlm/rag-prompt")

# 검색된 문서들을 하나의 텍스트로 결합하는 함수를 정의한다
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 종합부동산세 공제액 계산을 위한 RAG 체인을 구성한다
tax_deductible_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)


In [41]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# 종부세 공제액 계산을 위한 프롬프트 템플릿을 정의한다.
# Context에는 주택 수별 공제액 정보가 들어가고, Question에는 사용자의 질문이 들어간다.
# 프롬프트는 금액만 반환하도록 명확히 지시한다.
question = "10억짜리 집을 2채 가지고 있을 때 세금을 얼마나 내나요?"

user_deduction_prompt = """아래 [Context]는 주택에 대한 종합부동산세의 공제액에 관한 내용입니다.
사용자의 질문을 통해서 가지고 있는 주택수에 대한 공제액이 얼마인지 금액만 반환해주세요

[Context]
{tax_deductible_response}

[Question]
질문: {question}
답변:
"""

# PromptTemplate을 사용해 프롬프트의 변수 부분을 정의한다.
user_deduction_prompt_template = PromptTemplate(
   template=user_deduction_prompt,
   input_variables=['tax_deductible_response', 'question']
)

# 프롬프트 템플릿, LLM, 출력 파서를 연결하여 체인을 구성한다.
user_deduction_chain = (user_deduction_prompt_template
   | llm
   | StrOutputParser()
)

# 공제액 관련 질문을 정의하고 체인을 실행한다
deductible_question = "주택에 대한 종합부동산세 과세표준의 공제액을 알려주세요"
tax_deductible_response = '9억원'#tax_deductible_chain.invoke(deductible_question)

In [ ]:

from datetime import datetime
from langchain_community.tools import DuckDuckGoSearchRun

# 덕덕고 검색 도구를 초기화한다.
search = DuckDuckGoSearchRun()

# 현재 연도의 공정시장가액비율을 검색한다.
# datetime.now()로 현재 연도를 동적으로 가져와서 검색어에 포함한다.
market_value_rate_search = search.invoke(f"{datetime.now().year}년도 공정시장가액비율은?")

In [43]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 공정시장가액비율 추출을 위한 프롬프트 템플릿을 정의한다.
# Context에는 검색된 정보가 들어가고, 사용자의 질문을 바탕으로 해당하는 비율만 반환한다.
# 프롬프트는 부가 설명 없이 비율만 반환하도록 명확히 지시한다.
market_value_rate_prompt = PromptTemplate.from_template("""아래 [Context]는 공정시장가액비율에 관한 내용입니다.
당신에게 주어진 공정시장가액비율에 관한 내용을 기반으로, 사용자의 상황에 대한 공정시장가액비율을 알려주세요.
별도의 설명 없이 공정시장가액비율만 알려주세요

[Context]
{context}

[Question]
질문: {question}
답변:
""")

# 프롬프트 템플릿, LLM, 출력 파서를 연결하여 체인을 구성한다.
market_value_rate_chain = (
   market_value_rate_prompt
   | llm
   | StrOutputParser()
)

# 체인을 실행하여 검색 결과에서 사용자 상황에 맞는 공정시장가액비율을 추출한다.
market_value_rate =  '60%'#market_value_rate_chain.invoke({'context': market_value_rate_search, 'question': question})

In [10]:
from langchain_core.prompts import ChatPromptTemplate

# 챗봇 형식의 프롬프트 템플릿을 생성한다.
# system 메시지에는 세금 계산에 필요한 모든 기준 정보를 포함한다.
# human 메시지에는 사용자의 질문이 들어간다.
house_tax_prompt = ChatPromptTemplate.from_messages([
   ('system', f'''과세표준 계산방법: {tax_base_response}
공정시장가액비율: {market_value_rate}
공제액: {tax_deductible_response}

위의 공식과 아래 세율에 관한 정보를 활용해서 세금을 계산해주세요.
세율: {{tax_rate}}
'''),
   ('human', '{question}')
])

house_tax_chain = (
   {
       'tax_rate': retriever | format_docs,
       'question': RunnablePassthrough()
   }
   | house_tax_prompt
   | llm
   | StrOutputParser()
)

# 체인을 실행하여 최종 세금을 계산한다.
house_tax = house_tax_chain.invoke(question)
house_tax


'10억 원짜리 집을 2채 가지고 있을 경우, 총 자산 가치는 20억 원입니다. 이 경우, 2주택 이하를 소유한 경우의 세율을 적용합니다.\n\n1. 공제액: 2주택 이하의 경우 공제액은 9억 원입니다.\n2. 과세표준 계산:\n   - 공정시장가액비율: 60%\n   - 과세표준 = (20억 원 - 9억 원) * 60% = 11억 원 * 60% = 6.6억 원\n\n3. 세율 적용:\n   - 6억 원 초과 12억 원 이하 구간에 해당합니다.\n   - 세율: 360만 원 + (6억 원을 초과하는 금액의 1천분의 10)\n   - 초과 금액: 6.6억 원 - 6억 원 = 0.6억 원\n   - 세금 = 360만 원 + (0.6억 원 * 0.1) = 360만 원 + 60만 원 = 420만 원\n\n따라서, 10억 원짜리 집을 2채 가지고 있을 때 내야 할 세금은 420만 원입니다.'

In [44]:
from langchain_core.prompts import ChatPromptTemplate

# 챗봇 형식의 프롬프트 템플릿을 생성한다.
# system 메시지에는 세금 계산에 필요한 모든 기준 정보를 포함한다.
# human 메시지에는 사용자의 질문이 들어간다.
house_tax_prompt = ChatPromptTemplate.from_messages([
   ('system', f'''과세표준 계산방법: {tax_base_response}
공정시장가액비율: {market_value_rate}
공제액: {tax_deductible_response}

위의 공식과 아래 세율에 관한 정보를 활용해서 세금을 계산해주세요.
세율: {{tax_rate}}
'''),
   ('human', '{question}')
])

house_tax_chain = (
   {
       'tax_rate': retriever | format_docs,
       'question': RunnablePassthrough()
   }
   | house_tax_prompt
   | llm
   | StrOutputParser()
)

# 체인을 실행하여 최종 세금을 계산한다.
house_tax = house_tax_chain.invoke(question)
house_tax


'10억원짜리 집을 2채 가지고 있을 때 종합부동산세를 계산해드리겠습니다.\n\n**계산 과정:**\n\n1. **공시가격 합계액**: 10억원 × 2채 = 20억원\n\n2. **과세표준 계산**:\n   - (20억원 - 9억원) × 60% = 11억원 × 60% = 6.6억원\n\n3. **세율 적용**:\n   종합부동산세는 누진세율이 적용되는데, 주택의 경우 과세표준에 따라 다음과 같은 세율이 적용됩니다:\n   - 3억원 이하: 0.5%\n   - 3억원 초과 6억원 이하: 0.7%\n   - 6억원 초과 12억원 이하: 1.0%\n\n4. **세액 계산**:\n   - 3억원까지: 3억원 × 0.5% = 150만원\n   - 3억원 초과 6억원까지: 3억원 × 0.7% = 210만원\n   - 6억원 초과 6.6억원까지: 0.6억원 × 1.0% = 60만원\n   \n   **총 종합부동산세**: 150만원 + 210만원 + 60만원 = **420만원**\n\n따라서 10억원짜리 집을 2채 보유하신 경우, 연간 약 **420만원**의 종합부동산세를 납부하셔야 합니다.\n\n※ 이는 기본적인 계산이며, 실제로는 세부담 상한, 1세대 1주택 장기보유 공제 등 추가적인 요소들이 적용될 수 있습니다.'